# কার্টপোল ব্যালেন্সিংয়ের জন্য RL প্রশিক্ষণ

এই নোটবুকটি [AI for Beginners Curriculum](http://aka.ms/ai-beginners)-এর অংশ। এটি [PyTorch-এর অফিসিয়াল টিউটোরিয়াল](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) এবং [এই কার্টপোল PyTorch ইমপ্লিমেন্টেশন](https://github.com/yc930401/Actor-Critic-pytorch) দ্বারা অনুপ্রাণিত হয়েছে।

এই উদাহরণে, আমরা RL ব্যবহার করে একটি মডেল প্রশিক্ষণ করব যাতে একটি কার্টের উপর একটি পোল ব্যালেন্স করা যায়। কার্টটি অনুভূমিক স্কেলে বাম এবং ডান দিকে সরতে পারে। আমরা [OpenAI Gym](https://www.gymlibrary.ml/) পরিবেশ ব্যবহার করব পোলের সিমুলেশন করার জন্য।

> **Note**: আপনি এই পাঠের কোড লোকালভাবে (যেমন Visual Studio Code থেকে) চালাতে পারেন, সেক্ষেত্রে সিমুলেশনটি একটি নতুন উইন্ডোতে খুলবে। অনলাইনে কোড চালানোর সময়, কোডে কিছু পরিবর্তন করতে হতে পারে, যেমনটি [এখানে](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) বর্ণনা করা হয়েছে।

আমরা Gym ইনস্টল করা নিশ্চিত করার মাধ্যমে শুরু করব:


In [ ]:
import sys
!{sys.executable} -m pip install gym

এখন চলুন CartPole পরিবেশ তৈরি করি এবং এটি কীভাবে পরিচালনা করতে হয় তা দেখি। একটি পরিবেশের নিম্নলিখিত বৈশিষ্ট্যগুলি রয়েছে:

* **অ্যাকশন স্পেস** হলো সম্ভাব্য ক্রিয়াগুলির সেট যা আমরা সিমুলেশনের প্রতিটি ধাপে সম্পাদন করতে পারি  
* **অবজারভেশন স্পেস** হলো সেই পর্যবেক্ষণগুলির স্থান যা আমরা করতে পারি  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

চলুন দেখি সিমুলেশনটি কীভাবে কাজ করে। নিচের লুপটি সিমুলেশন চালায়, যতক্ষণ না `env.step` টার্মিনেশন ফ্ল্যাগ `done` ফেরত দেয়। আমরা এলোমেলোভাবে অ্যাকশন নির্বাচন করব `env.action_space.sample()` ব্যবহার করে, যার মানে পরীক্ষাটি সম্ভবত খুব দ্রুত ব্যর্থ হবে (CartPole পরিবেশটি তখনই শেষ হয় যখন CartPole-এর গতি, অবস্থান বা কোণ নির্দিষ্ট সীমার বাইরে চলে যায়)।

> সিমুলেশনটি একটি নতুন উইন্ডোতে খুলবে। আপনি কোডটি একাধিকবার চালাতে পারেন এবং এটি কীভাবে আচরণ করে তা দেখতে পারেন।


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

আপনি লক্ষ্য করবেন যে পর্যবেক্ষণগুলোতে ৪টি সংখ্যা রয়েছে। সেগুলো হলো:
- কার্টের অবস্থান
- কার্টের বেগ
- পোলের কোণ
- পোলের ঘূর্ণনের হার

`rew` হলো সেই পুরস্কার যা আমরা প্রতিটি ধাপে পাই। CartPole পরিবেশে আপনি প্রতিটি সিমুলেশন ধাপের জন্য ১ পয়েন্ট পুরস্কৃত হন, এবং লক্ষ্য হলো মোট পুরস্কার সর্বাধিক করা, অর্থাৎ CartPole যতক্ষণ পর্যন্ত না পড়ে যায় ততক্ষণ এটি ভারসাম্য বজায় রাখতে সক্ষম হওয়া।

রিইনফোর্সমেন্ট লার্নিংয়ের সময়, আমাদের লক্ষ্য হলো একটি **পলিসি** $\pi$ প্রশিক্ষণ দেওয়া, যা প্রতিটি অবস্থার $s$ জন্য আমাদের জানাবে কোন ক্রিয়া $a$ গ্রহণ করতে হবে, অর্থাৎ মূলত $a = \pi(s)$।

যদি আপনি একটি প্রোবাবিলিস্টিক সমাধান চান, তবে আপনি পলিসিকে প্রতিটি ক্রিয়ার জন্য সম্ভাবনার একটি সেট প্রদানকারী হিসেবে ভাবতে পারেন, অর্থাৎ $\pi(a|s)$ বোঝাবে যে আমরা অবস্থান $s$-এ ক্রিয়া $a$ গ্রহণ করার সম্ভাবনা কত।

## পলিসি গ্রেডিয়েন্ট পদ্ধতি

সবচেয়ে সহজ RL অ্যালগরিদমে, যাকে **পলিসি গ্রেডিয়েন্ট** বলা হয়, আমরা একটি নিউরাল নেটওয়ার্ক প্রশিক্ষণ দেবো যাতে এটি পরবর্তী ক্রিয়া পূর্বানুমান করতে পারে।


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

আমরা নেটওয়ার্কটি প্রশিক্ষণ দেব অনেক পরীক্ষা চালিয়ে, এবং প্রতিটি চালানোর পরে আমাদের নেটওয়ার্ক আপডেট করব। চলুন একটি ফাংশন সংজ্ঞায়িত করি যা পরীক্ষা চালাবে এবং ফলাফলগুলি (যাকে **ট্রেস** বলা হয়) ফিরিয়ে দেবে - সমস্ত অবস্থা, ক্রিয়া (এবং তাদের সুপারিশকৃত সম্ভাবনা), এবং পুরস্কার:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

আপনি প্রশিক্ষণবিহীন নেটওয়ার্ক দিয়ে একটি পর্ব চালাতে পারেন এবং লক্ষ্য করতে পারেন যে মোট পুরস্কার (অর্থাৎ পর্বের দৈর্ঘ্য) খুব কম:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

পলিসি গ্রেডিয়েন্ট অ্যালগরিদমের একটি জটিল দিক হল **ডিসকাউন্টেড রিওয়ার্ডস** ব্যবহার করা। ধারণাটি হল যে আমরা গেমের প্রতিটি ধাপে মোট রিওয়ার্ডের ভেক্টর গণনা করি, এবং এই প্রক্রিয়ার সময় আমরা কিছু সহগ $gamma$ ব্যবহার করে প্রাথমিক রিওয়ার্ডগুলো ডিসকাউন্ট করি। আমরা ফলাফলস্বরূপ ভেক্টরটিকে স্বাভাবিকীকরণও করি, কারণ আমরা এটি আমাদের প্রশিক্ষণে প্রভাবিত করার ওজন হিসেবে ব্যবহার করব।


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

এখন আমরা প্রকৃত প্রশিক্ষণ শুরু করব! আমরা ৩০০টি এপিসোড চালাব, এবং প্রতিটি এপিসোডে আমরা নিম্নলিখিত কাজগুলো করব:

1. পরীক্ষাটি চালান এবং ট্রেস সংগ্রহ করুন।
1. নেওয়া পদক্ষেপ এবং পূর্বাভাসিত সম্ভাবনার মধ্যে পার্থক্য (`gradients`) গণনা করুন। পার্থক্য যত কম হবে, আমরা তত বেশি নিশ্চিত হতে পারব যে সঠিক পদক্ষেপ নেওয়া হয়েছে।
1. ডিসকাউন্টেড রিওয়ার্ডস গণনা করুন এবং ডিসকাউন্টেড রিওয়ার্ডস দ্বারা গ্রেডিয়েন্টসকে গুণ করুন - এটি নিশ্চিত করবে যে উচ্চ রিওয়ার্ডযুক্ত পদক্ষেপগুলো চূড়ান্ত ফলাফলে কম রিওয়ার্ডযুক্ত পদক্ষেপগুলোর তুলনায় বেশি প্রভাব ফেলবে।
1. আমাদের নিউরাল নেটওয়ার্কের জন্য প্রত্যাশিত লক্ষ্য পদক্ষেপ আংশিকভাবে রান চলাকালীন পূর্বাভাসিত সম্ভাবনা থেকে এবং আংশিকভাবে গণিত করা গ্রেডিয়েন্টস থেকে নেওয়া হবে। আমরা `alpha` প্যারামিটার ব্যবহার করব গ্রেডিয়েন্টস এবং রিওয়ার্ডসকে কতটা বিবেচনা করা হবে তা নির্ধারণ করতে - এটিকে *রিইনফোর্সমেন্ট অ্যালগরিদমের* শেখার হার বলা হয়।
1. শেষ পর্যন্ত, আমরা আমাদের নেটওয়ার্ককে স্টেটস এবং প্রত্যাশিত পদক্ষেপগুলোর উপর প্রশিক্ষণ দেব এবং প্রক্রিয়াটি পুনরাবৃত্তি করব।


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

এখন চলুন পর্বটি রেন্ডারিং সহ চালাই এবং ফলাফল দেখি:


In [ ]:
_ = run_episode(render=True)

আশা করি, আপনি দেখতে পাচ্ছেন যে এখন পোলটি বেশ ভালোভাবে ব্যালেন্স করতে পারছে!

## অ্যাক্টর-ক্রিটিক মডেল

অ্যাক্টর-ক্রিটিক মডেল হলো পলিসি গ্রেডিয়েন্টের আরও উন্নত সংস্করণ, যেখানে আমরা একটি নিউরাল নেটওয়ার্ক তৈরি করি যা পলিসি এবং অনুমানকৃত রিওয়ার্ড উভয়ই শিখতে পারে। এই নেটওয়ার্কের দুটি আউটপুট থাকবে (অথবা আপনি এটিকে দুটি পৃথক নেটওয়ার্ক হিসেবে দেখতে পারেন):
* **অ্যাক্টর** আমাদের স্টেট প্রোবাবিলিটি ডিস্ট্রিবিউশন দিয়ে অ্যাকশন নেওয়ার সুপারিশ করবে, যেমনটি পলিসি গ্রেডিয়েন্ট মডেলে হয়।
* **ক্রিটিক** অনুমান করবে যে ঐ অ্যাকশনগুলো থেকে রিওয়ার্ড কী হতে পারে। এটি প্রদত্ত স্টেটে ভবিষ্যতে মোট অনুমানকৃত রিওয়ার্ড প্রদান করবে।

চলুন এমন একটি মডেল সংজ্ঞায়িত করি:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

আমাদের `discounted_rewards` এবং `run_episode` ফাংশনগুলোকে সামান্য পরিবর্তন করতে হবে:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


এখন আমরা প্রধান প্রশিক্ষণ লুপ চালাব। আমরা সঠিক ক্ষতি ফাংশন গণনা করে এবং নেটওয়ার্ক প্যারামিটার আপডেট করে ম্যানুয়াল নেটওয়ার্ক প্রশিক্ষণ প্রক্রিয়া ব্যবহার করব:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## মূল বিষয়

আমরা এই ডেমোতে দুটি RL অ্যালগরিদম দেখেছি: সাধারণ পলিসি গ্রেডিয়েন্ট এবং আরও উন্নত অ্যাক্টর-ক্রিটিক। আপনি দেখতে পাবেন যে এই অ্যালগরিদমগুলো অবস্থা, ক্রিয়া এবং পুরস্কারের বিমূর্ত ধারণার সাথে কাজ করে - তাই এগুলো খুব ভিন্ন পরিবেশেও প্রয়োগ করা যেতে পারে।

রিইনফোর্সমেন্ট লার্নিং আমাদেরকে শুধুমাত্র চূড়ান্ত পুরস্কার দেখে সমস্যার সেরা কৌশল শিখতে সাহায্য করে। লেবেলযুক্ত ডেটাসেটের প্রয়োজন না হওয়ার কারণে আমরা আমাদের মডেলগুলোকে অপ্টিমাইজ করার জন্য সিমুলেশনগুলো বারবার চালাতে পারি। তবে, RL-এ এখনও অনেক চ্যালেঞ্জ রয়েছে, যা আপনি শিখতে পারবেন যদি আপনি AI-এর এই আকর্ষণীয় ক্ষেত্রে আরও গভীরভাবে মনোযোগ দেওয়ার সিদ্ধান্ত নেন।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
